# **Demo: Model Fine-tuning with Fireworks' LoRA**

## **🎯 Overview**

This demo shows how to use **Fireworks' LoRA fine-tuning** to customize a model for your specific use case. LoRA (Low-Rank Adaptation) is an efficient fine-tuning technique that allows you to adapt large language models while:
- 🚀 Training fewer parameters
- 💾 Using less memory
- ✨ Maintaining model quality

We'll fine-tune a model on a customer support dataset to create an AI assistant that can handle support queries.

## **🔑 Prerequisites**

1. A Fireworks AI account (Sign up [here](https://fireworks.ai/login))
2. API key and Account ID:
   - Log into your Fireworks account
   - Click on the Profile Icon -> API Keys
   - Create a new API key if you don't have one
   - Note your Account ID beside the Profile Icon

## **📚 Understanding LoRA Fine-tuning**

Before we begin, let's understand what makes a good fine-tuning project:

1. **When to use LoRA:**
   - You want to specialize a model for a specific domain
   - You have high-quality, consistent training data
   - You need faster training and lower resource usage than full fine-tuning

## **📝 Steps**

1. 🛠️ Setup and Installation
2. 📊 Prepare the Dataset
3. ⚡ Fine-tune the Model
4. 🧪 Test and Evaluate
5. 🧹 Clean Up

Let's get started!

## 1. **🛠️ Setup and Installation**

In [ ]:
# Install the Fireworks and Datasets libraries
!pip install fireworks-ai datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: anyio
    Found existing installation: anyio 3.7.1
    Uninstalling anyio-3.7.1:
      Successfully uninstalled anyio-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [ ]:
# Import Libraries
from google.colab import drive
from datasets import load_dataset
import requests
import os
import json

In [ ]:
# Configure Fireworks credentials
API_KEY = "YOUR_API_KEY"  # Replace with your API key
ACCOUNT_ID = "YOUR_ACCOUNT_ID"  # Replace with your account ID
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
HEADERS_WITH_CONTENT_TYPE = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Project configuration
BASE_URL = f"https://api.fireworks.ai/v1/accounts/{ACCOUNT_ID}"
BASE_MODEL = "llama-v3p1-8b-instruct"
DATASET_ID = "customer-support-demo"
OUTPUT_MODEL = "customer-support-assistant"

## 2. **📊 Prepare the Dataset**

### Understanding the Data Format

The dataset schema needs to conform to the OpenAI-compatible Chat Completions API. Each example should be a JSON object with:
- A system message defining the assistant's role
- A user message (input)
- An assistant message (desired output)

Here's what good training data looks like:

```python
{
    "messages": [
        {"role": "system", "content": "You are a helpful customer support assistant."},
        {"role": "user", "content": "How do I reset my password?"},
        {"role": "assistant", "content": "I can help you reset your password. Please follow these steps..."}
    ]
}
```

### Loading and Formatting Data

In [ ]:

# Load example customer support dataset
# Dataset from: https://huggingface.co/datasets/bitext/Bitext-customer-support-llm-chatbot-training-dataset
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

# Convert to Fireworks' expected format
def format_example(example):
    return {
        "messages": [
            {"role": "system", "content": "You are a helpful customer support assistant."},
            {"role": "user", "content": example["instruction"]}, # 'the data labels may vary for each dataset
            {"role": "assistant", "content": example["response"]}
        ]
    }

# Process dataset and save as JSONL
with open("support_dataset.jsonl", "w") as f:
    for example in dataset["train"]:
        json_line = format_example(example)
        f.write(json.dumps(json_line) + "\n")

print("Dataset preview:")
!head -n 2 support_dataset.jsonl

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

(…)t_Training_Dataset_27K_responses-v11.csv:   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

Dataset preview:
{"messages": [{"role": "system", "content": "You are a helpful customer support assistant."}, {"role": "user", "content": "question about cancelling order {{Order Number}}"}, {"role": "assistant", "content": "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}]}
{"messages": [{"role": "system", "content": "You are a helpful customer support assistant."}, {"role": "user", "content": "i have a question about cancelling oorder {{Order Number}}"}, {"role": "assistant", "content": "I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority."}]}


This works on Google CoLab (a linux-based machine) but if you're running this on Windows/Mac OS, please check out how to [install firectl](https://docs.fireworks.ai/tools-sdks/firectl/firectl).

In [ ]:
# install firectl
!wget -O firectl.gz https://storage.googleapis.com/fireworks-public/firectl/stable/linux-amd64.gz
!gunzip firectl.gz
!sudo install -o root -g root -m 0755 firectl /usr/local/bin/firectl

--2025-01-27 18:31:26--  https://storage.googleapis.com/fireworks-public/firectl/stable/linux-amd64.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17197919 (16M) [application/octet-stream]
Saving to: ‘firectl.gz’

firectl.gz          100%[===================>]  16.40M  --.-KB/s    in 0.1s    

2025-01-27 18:31:26 (111 MB/s) - ‘firectl.gz’ saved [17197919/17197919]



In [ ]:
!firectl set-api-key {API_KEY} -a {ACCOUNT_ID}

## 3. **⚡ Fine-tune the Model**


In [ ]:
# First, create the dataset on Fireworks
!firectl -a {ACCOUNT_ID} create dataset {DATASET_ID} support_dataset.jsonl

# Let's explore the key tuning parameters:

# Basic fine-tuning (using defaults)
print("Starting basic fine-tuning...")
!firectl -a {ACCOUNT_ID} create sftj \
    --base-model {BASE_MODEL} \
    --dataset {DATASET_ID} \
    --output-model {OUTPUT_MODEL}

2025/01/24 23:36:12 Failed to execute: error creating dataset: rpc error: code = AlreadyExists desc = dataset with the same name already exists
Starting basic fine-tuning...
Name: accounts/fireworks/supervisedFineTuningJobs/8b037a38
Create Time: 2025-01-24 23:36:12
Dataset: accounts/fireworks/datasets/customer-support-demo
State: JOB_STATE_VALIDATING
Status: OK
Created By: andre@fireworks.ai
Output Model: accounts/fireworks/models/customer-support-assistant
Base Model: accounts/fireworks/models/llama-v3p1-8b-instruct
Learning Rate: 0
Max Context Length: 8192
Lora Rank: 8
Wandb Config:


Key Parameters Explained:
------------------------
- epochs: Number of training passes (default=1)
    - Increase if model needs more training (max 3)
    - Example: --epochs 2

- learning-rate: Controls update speed (default is model-specific)
    - Usually best to keep default
    - Example: --learning-rate 0.0001

- lora-rank: Number of trainable parameters (default=8)
    - Must be power of 2, up to 64
    - Higher = more capacity but slower
    - Example: --lora-rank 16

- early-stop: Stops if not improving
    - Optional flag
    - Example: --early-stop

- max-context-length: Maximum input length
    - Default is 8k
    - Example: --max-context-length 16000

In [ ]:
config = {
    # LoRA specific settings
    'lora_rank': 8,                           # Must be power of 2, up to 64 (default: 8) increase to increase the amount of information captured while tuning
    'learning_rate': 1e-4,                    # recommended not to change
    'epochs': 1,                              # increase if not following training data as expected

    # Optional settings
    'max_context_length': 8192,
    'early_stop': False,
    # 'evaluation_dataset': 'my_eval_dataset' # You may upload your own evaluation dataset
}

In [ ]:
# Example with custom parameters
!firectl create sftj \
    --base-model {BASE_MODEL} \
    --dataset {DATASET_ID} \
    --output-model {OUTPUT_MODEL}-custom \
    --epochs {config['epochs']} \
    --learning-rate {config['learning_rate']} \
    --lora-rank {config['lora_rank']} \
    --max-context-length {config['max_context_length']}\
    {'--early-stop' if config['early_stop'] else ''}\
    -a {ACCOUNT_ID}


Name: accounts/fireworks/supervisedFineTuningJobs/7caa8cc0
Create Time: 2025-01-24 23:55:06
Dataset: accounts/fireworks/datasets/customer-support-demo
State: JOB_STATE_VALIDATING
Status: OK
Created By: andre@fireworks.ai
Output Model: accounts/fireworks/models/customer-support-assistant-custom
Base Model: accounts/fireworks/models/llama-v3p1-8b-instruct
Epochs: 1
Learning Rate: 0.0001
Max Context Length: 8192
Lora Rank: 8
Wandb Config:
2025/01/24 23:55:06 Failed to execute: rpc error: code = NotFound desc = resource not found


You can access the fine tuning job ID from your account's [deployments](https://fireworks.ai/dashboard/fine-tuning). This lets you monitor the training progress of the job.

In [ ]:
# Monitor training progress
!firectl -a {ACCOUNT_ID} get supervised-fine-tuning-job 7caa8cc0

Name: accounts/fireworks/supervisedFineTuningJobs/7caa8cc0
Create Time: 2025-01-24 23:55:06
Dataset: accounts/fireworks/datasets/customer-support-demo
State: JOB_STATE_COMPLETED
Status: OK
Created By: andre@fireworks.ai
Output Model: accounts/fireworks/models/customer-support-assistant-custom
Base Model: accounts/fireworks/models/llama-v3p1-8b-instruct
Epochs: 1
Learning Rate: 0.0001
Max Context Length: 8192
Lora Rank: 8
Wandb Config:


## 4. 🧪 Deploy and Test the Fine-tuned Model

Deploy the model so that we can test it.
On Fireworks, we support serverless LoRA addons for [these models](https://docs.fireworks.ai/fine-tuning/fine-tuning-models#supported-base-models-loras-on-serverless).

In [ ]:
# Deploy the model (required before inference)
!firectl -a {ACCOUNT_ID} deploy {OUTPUT_MODEL}-custom

In [ ]:
!firectl -a {ACCOUNT_ID} deploy {OUTPUT_MODEL}

In [ ]:
# Test with example queries

def test_model(query):
    """Test the fine-tuned model with a customer support query"""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": f"accounts/{ACCOUNT_ID}/models/{OUTPUT_MODEL}-custom",
        # "model": f"accounts/{ACCOUNT_ID}/models/{OUTPUT_MODEL}", you can test between the addon with adjusted parameters and the default fine-tuned LoRA
        "messages": [
            {
                "role": "user",
                "content": query
            }
        ]
    }

    response = requests.post(
        "https://api.fireworks.ai/inference/v1/chat/completions",
        headers=headers,
        data=json.dumps(payload)
    )

    return response.json()["choices"][0]["message"]["content"]

# Test examples
test_queries = [
    "How do I reset my password?",
    "What's your return policy?",
    "I haven't received my order yet"
]

print("Testing the fine-tuned model:")
for query in test_queries:
    print(f"\nQuery: {query}")
    print(f"Response: {test_model(query)}")

Testing the fine-tuned model:

Query: How do I reset my password?
Response: I'll get right on it! I'm here to guide you through the process of resetting your password.

1. **Access the Password Reset Page**: Start by visiting the login screen of our platform. You'll find a link to the password reset page, which will prompt you to enter your email address.
2. **Enter Your Email**: Enter the email address you used when you signed up for our platform.
3. **Click on the Reset Password Button**: Once you've entered your email address, click on the "Reset Password" button.
4. **Check Your Email**: Keep an eye on your email inbox, including the spam or junk folder, for a message titled "Password Reset" or something similar. If you don't see it within a few minutes, please check your phone or other email accounts as well.
5. **Click on the Reset Link**: Open the email and click on the "Reset Password" or "Click Here" link provided.
6. **Create a New Password**: Follow the instructions in the e

You may encounter an error stating that the model is inaccessible/undeployed. If that's the case, check out [your models](https://fireworks.ai/dashboard/models) to see the status of deployment.

# 5.  **🧹 Clean Up**

Now you can delete all resources you have created. The order of deletion does not matter, except the model needs to be undeployed before you can delete it.

In [ ]:
# Undeploy model
!firectl -a {ACCOUNT_ID} undeploy {OUTPUT_MODEL}
!firectl -a {ACCOUNT_ID} undeploy {OUTPUT_MODEL}-custom

In [ ]:
# Delete model (optional)
!firectl -a {ACCOUNT_ID} delete model {OUTPUT_MODEL}
!firectl -a {ACCOUNT_ID} delete model {OUTPUT_MODEL}-custom

In [ ]:
# Delete dataset
!firectl -a {ACCOUNT_ID} delete dataset {DATASET_ID}